# Pull Data

In [1]:
# add quantkit to path
import sys
from pathlib import Path

d = Path().resolve().parent.parent
sys.path.insert(0, str(d))

## FRED

In [2]:
import quantkit.handyman.fred_data_loader as fred_data_loader
import quantkit.handyman.bloomberg_data_loader as bloomberg_data_loader
import pandas as pd

# weekly data, 1, 3 month MA
jobless_claims = fred_data_loader.run_fred_api(
    "ICSA", revision=True, realtime_start="2007-01-01", averages=[4, 12]
)
jobless_claims["Source"] = "FRED"

# monthly data, 3 month MA
personal_income = fred_data_loader.run_fred_api(
    "PI", revision=True, realtime_start="2007-01-01", averages=[3]
)
personal_income["Source"] = "FRED"

# weekly data, 3 month MA, 1 year change
consumer_loans = fred_data_loader.run_fred_api(
    "CLSACBW027SBOG",
    revision=True,
    realtime_start="2007-01-01",
    averages=[12],
    changes=[52],
)
consumer_loans["Source"] = "FRED"

# weekly data, 3 month MA, 1 year change
real_estate_loans = fred_data_loader.run_fred_api(
    "RELACBW027SBOG",
    revision=True,
    realtime_start="2007-01-01",
    averages=[12],
    changes=[52],
)
real_estate_loans["Source"] = "FRED"

# monthly data, month over month difference
unemployment_rate = fred_data_loader.run_fred_api(
    "UNRATE", revision=True, realtime_start="2007-01-01", differences=[1]
)
unemployment_rate["Source"] = "FRED"

## Bloomberg

In [3]:
bb_list = [
    "NAPMPROD Index",
    "CONCCONF Index",
    "CPI YOY Index",
    "PCE CYOY Index",
    "JTSYBE5 Index",
    "JTSYBE10 Index",
    "FED5YEAR Index",
    "USSOC Curncy",
    "SHY US Equity",
    "IEF US Equity",
    "USGG10YR Index",
    "USGG2YR Index",
    "CLEV16%Y Index",
    "CLEVXPYC Index",
    "USGG5Y5Y INDEX",
    "S0042FS 3M3M BLC CURNCY",
    "JFRIUS INDEX",
    "FARWUSNB Index",
    "FARWUSII Index",
    "DEBPNOTE Index",
    "DEBPBOND Index",
    "DEBPINNT Index",
]

bb_d = dict()

for ind in bb_list:
    data = bloomberg_data_loader.get_price_data([ind], start_date="1/1/1980")
    data["Source"] = "Bloomberg"
    bb_d[ind] = data

# Write Default Data

In [3]:
with pd.ExcelWriter("economic_data.xlsx", engine="xlsxwriter") as writer:
    jobless_claims.to_excel(writer, sheet_name="Jobless Claims", index=False)
    personal_income.to_excel(writer, sheet_name="Personal Income", index=False)
    consumer_loans.to_excel(writer, sheet_name="Consumer Loans", index=False)
    real_estate_loans.to_excel(writer, sheet_name="Real Estata Loans", index=False)
    unemployment_rate.to_excel(writer, sheet_name="Unemployment Rate", index=False)
    for ind, data in bb_d.items():
        data.to_excel(writer, sheet_name=ind)

# Write Monday Data

In [4]:
from datetime import datetime
from dateutil import relativedelta

datetime(2023, 1, 1) + relativedelta.relativedelta(day=31, weekday=relativedelta.MO(-1))

years = [i for i in range(1980, 2025)]
months = [i for i in range(1, 13)]

last_mondays = list()

for year in years:
    for month in months:
        last_mondays.append(
            datetime(year, month, 1)
            + relativedelta.relativedelta(day=31, weekday=relativedelta.MO(-1))
        )

In [5]:
dfs = list()
fred_dfs = [
    jobless_claims,
    personal_income,
    consumer_loans,
    real_estate_loans,
    unemployment_rate,
]

for d in fred_dfs:
    d_temp = d.set_index("publish_date")
    d_temp = d_temp.sort_index()
    d_temp = d_temp.drop(["date_from", "Source"], axis=1)
    for col in d_temp.columns:
        l = list()
        for monday in last_mondays:
            val = d_temp.asof(monday)[col]
            l.append((monday, val))
        dfs.append(pd.DataFrame(l, columns=["Date", f"{col}"]).set_index("Date"))

for ind, data in bb_d.items():
    d_temp = data.drop(["Source"], axis=1)
    d_temp = d_temp.sort_index()
    for col in d_temp.columns:
        l = list()
        for monday in last_mondays:
            val = d_temp.asof(monday)[col]
            l.append((monday, val))
        dfs.append(pd.DataFrame(l, columns=["Date", f"{col}"]).set_index("Date"))

In [6]:
monday_data = pd.concat([i for i in dfs], axis=1)

# Calculte Z-Scores

In [7]:
dfs = list()

for d in fred_dfs:
    d_temp = d.set_index("publish_date")
    d_temp = d_temp.drop(["date_from", "Source"], axis=1)
    for col in d_temp.columns:
        l = list()

        z_score = (d_temp[col] - d_temp[col].mean()) / d_temp[col].std()
        l.append((col, z_score.values[-1]))
        dfs.append(pd.DataFrame(l, columns=["Factor", f"Z-Score"]).set_index("Factor"))

for ind, data in bb_d.items():
    d_temp = data.drop(["Source"], axis=1)
    for col in d_temp.columns:
        l = list()

        z_score = (d_temp[col] - d_temp[col].mean()) / d_temp[col].std()
        l.append((col, z_score.values[-1]))
        dfs.append(pd.DataFrame(l, columns=["Factor", f"Z-Score"]).set_index("Factor"))

In [8]:
z_score_data = pd.concat([i for i in dfs])

# Write Data Dictionary

In [11]:
mapping_dict = [
    (
        "ICSA",
        "Unemployment Insurance Claims Report",
        "FRED",
        "Weekly",
        jobless_claims["date_from"].min(),
        jobless_claims["date_from"].max(),
    ),
    (
        "ICSA_4MA",
        "Unemployment Insurance Claims Report 1 Month Moving Average",
        "Calculated",
        "Weekly",
        jobless_claims["date_from"].min(),
        jobless_claims["date_from"].max(),
    ),
    (
        "ICSA_12MA",
        "Unemployment Insurance Claims Report 3 Months Moving Average",
        "Calculated",
        "Weekly",
        jobless_claims["date_from"].min(),
        jobless_claims["date_from"].max(),
    ),
    (
        "PI",
        "Personal Income",
        "FRED",
        "Monthly",
        personal_income["date_from"].min(),
        personal_income["date_from"].max(),
    ),
    (
        "PI_3MA",
        "Personal Income 3 Months Moving Average",
        "Calculated",
        "Monthly",
        personal_income["date_from"].min(),
        personal_income["date_from"].max(),
    ),
    (
        "CLSACBW027SBOG",
        "Consumer Loans, All Commercial Banks",
        "FRED",
        "Weekly",
        consumer_loans["date_from"].min(),
        consumer_loans["date_from"].max(),
    ),
    (
        "CLSACBW027SBOG_12MA",
        "Consumer Loans, All Commercial Banks 3 Months Moving Average",
        "Calculated",
        "Weekly",
        consumer_loans["date_from"].min(),
        consumer_loans["date_from"].max(),
    ),
    (
        "CLSACBW027SBOG_52Change",
        "Consumer Loans, All Commercial Banks 1 Year % Change",
        "Calculated",
        "Weekly",
        consumer_loans["date_from"].min(),
        consumer_loans["date_from"].max(),
    ),
    (
        "RELACBW027SBOG",
        "Real Estate Loans, All Commercial Banks",
        "FRED",
        "Weekly",
        real_estate_loans["date_from"].min(),
        real_estate_loans["date_from"].max(),
    ),
    (
        "RELACBW027SBOG_12MA",
        "Real Estate Loans, All Commercial Banks 3 Months Moving Average",
        "Calculated",
        "Weekly",
        real_estate_loans["date_from"].min(),
        real_estate_loans["date_from"].max(),
    ),
    (
        "RELACBW027SBOG_52Change",
        "Real Estate Loans, All Commercial Banks 1 Year % Change",
        "Calculated",
        "Weekly",
        real_estate_loans["date_from"].min(),
        real_estate_loans["date_from"].max(),
    ),
    (
        "UNRATE",
        "Unemployment Rate",
        "FRED",
        "Monthly",
        unemployment_rate["date_from"].min(),
        unemployment_rate["date_from"].max(),
    ),
    (
        "UNRATE_1Diff",
        "Unemployment Rate Month over Month Change",
        "Calculated",
        "Monthly",
        unemployment_rate["date_from"].min(),
        unemployment_rate["date_from"].max(),
    ),
    (
        "NAPMPROD Index",
        "ISM Manufacturing Report on Business Production SA",
        "Bloomberg",
        "Monthly",
        bb_d["NAPMPROD Index"].index.min(),
        bb_d["NAPMPROD Index"].index.max(),
    ),
    (
        "CONCCONF Index",
        "Conference Board Consumer Confidence SA",
        "Bloomberg",
        "Monthly",
        bb_d["CONCCONF Index"].index.min(),
        bb_d["CONCCONF Index"].index.max(),
    ),
    (
        "CPI YOY Index",
        "US CPI Urban Consumers YoY NSA",
        "Bloomberg",
        "Monthly",
        bb_d["CPI YOY Index"].index.min(),
        bb_d["CPI YOY Index"].index.max(),
    ),
    (
        "PCE CYOY Index",
        "US Personal Consumption Expenditure Core Price Index YoY SA",
        "Bloomberg",
        "Monthly",
        bb_d["PCE CYOY Index"].index.min(),
        bb_d["PCE CYOY Index"].index.max(),
    ),
    (
        "JTSYBE5 Index",
        "J.P. Morgan 5-year TIPS Breakeven Fair Value model",
        "Bloomberg",
        "Daily",
        bb_d["JTSYBE5 Index"].index.min(),
        bb_d["JTSYBE5 Index"].index.max(),
    ),
    (
        "JTSYBE10 Index",
        "J.P. Morgan 10-year TIPS Breakeven Fair Value model",
        "Bloomberg",
        "Daily",
        bb_d["JTSYBE10 Index"].index.min(),
        bb_d["JTSYBE10 Index"].index.max(),
    ),
    (
        "FED5YEAR Index",
        "US Federal Reserve Five-year Forward Five Year Breakeven Inflation Rate",
        "Bloomberg",
        "Daily",
        bb_d["FED5YEAR Index"].index.min(),
        bb_d["FED5YEAR Index"].index.max(),
    ),
    (
        "USSOC Curncy",
        "USD SWAP OIS",
        "Bloomberg",
        "Daily",
        bb_d["USSOC Curncy"].index.min(),
        bb_d["USSOC Curncy"].index.max(),
    ),
    (
        "SHY US Equity",
        "ISHARES 1-3 YEAR TREASURY BO",
        "Bloomberg",
        "Daily",
        bb_d["SHY US Equity"].index.min(),
        bb_d["SHY US Equity"].index.max(),
    ),
    (
        "IEF US Equity",
        "ISHARES 7-10 YEAR TREASURY B",
        "Bloomberg",
        "Daily",
        bb_d["IEF US Equity"].index.min(),
        bb_d["IEF US Equity"].index.max(),
    ),
    (
        "USGG10YR Index",
        "US Generic Govt 10 Yr",
        "Bloomberg",
        "Daily",
        bb_d["USGG10YR Index"].index.min(),
        bb_d["USGG10YR Index"].index.max(),
    ),
    (
        "USGG2YR Index",
        "US Generic Govt 2 Yr",
        "Bloomberg",
        "Daily",
        bb_d["USGG2YR Index"].index.min(),
        bb_d["USGG2YR Index"].index.max(),
    ),
    (
        "CLEV16%Y Index",
        "Federal Reserve Bank of Cleveland 16% Trimmed-Mean CPI YoY% Change",
        "Bloomberg",
        "Monthly",
        bb_d["CLEV16%Y Index"].index.min(),
        bb_d["CLEV16%Y Index"].index.max(),
    ),
    (
        "CLEVXPYC Index",
        "US Cleveland Fed Inflation Nowcasting Core PCE YoY Current Period",
        "Bloomberg",
        "Daily",
        bb_d["CLEVXPYC Index"].index.min(),
        bb_d["CLEVXPYC Index"].index.max(),
    ),
    (
        "USGG5Y5Y INDEX",
        "U.S. 5yr 5yr Forward Breakeven",
        "Bloomberg",
        "Daily",
        bb_d["USGG5Y5Y INDEX"].index.min(),
        bb_d["USGG5Y5Y INDEX"].index.max(),
    ),
    (
        "S0042FS 3M3M BLC CURNCY",
        "USD OIS FWD swap 3M3M",
        "Bloomberg",
        "Daily",
        bb_d["S0042FS 3M3M BLC CURNCY"].index.min(),
        bb_d["S0042FS 3M3M BLC CURNCY"].index.max(),
    ),
    (
        "JFRIUS INDEX",
        "JPMorgan Forecast Revision Index United States",
        "Bloomberg",
        "Weekly",
        bb_d["JFRIUS INDEX"].index.min(),
        bb_d["JFRIUS INDEX"].index.max(),
    ),
    (
        "FARWUSNB Index",
        "Reserve Balance Wednesday Close Notes & Bonds Held",
        "Bloomberg",
        "Weekly",
        bb_d["FARWUSNB Index"].index.min(),
        bb_d["FARWUSNB Index"].index.max(),
    ),
    (
        "FARWUSII Index",
        "Reserve Balance Wednesday Close Inflation Indexed",
        "Bloomberg",
        "Weekly",
        bb_d["FARWUSII Index"].index.min(),
        bb_d["FARWUSII Index"].index.max(),
    ),
    (
        "DEBPNOTE Index",
        "US Total Debt Outstanding Notes",
        "Bloomberg",
        "Monthly",
        bb_d["DEBPNOTE Index"].index.min(),
        bb_d["DEBPNOTE Index"].index.max(),
    ),
    (
        "DEBPBOND Index",
        "US Total Debt Outstanding Bonds",
        "Bloomberg",
        "Monthly",
        bb_d["DEBPBOND Index"].index.min(),
        bb_d["DEBPBOND Index"].index.max(),
    ),
    (
        "DEBPINNT Index",
        "US Total Debt Outstanding TIPS",
        "Bloomberg",
        "Monthly",
        bb_d["DEBPINNT Index"].index.min(),
        bb_d["DEBPINNT Index"].index.max(),
    ),
]

desc_df = pd.DataFrame(
    mapping_dict,
    columns=["Factor", "Description", "Source", "Frequency", "Start Date", "End Date"],
)

In [20]:
with pd.ExcelWriter("Monday_Economic_Data.xlsx", engine="xlsxwriter") as writer:
    monday_data.to_excel(writer, sheet_name="Data")
    z_score_data.to_excel(writer, sheet_name="Z-Scores")
    desc_df.to_excel(writer, sheet_name="Data Dictionary", index=False)

: 